In [4]:
import os, csv
from itertools import islice

from numba import cuda
cuda.select_device(5)

import matplotlib.pyplot as plt
import numpy as np

import nibabel as nib
import pandas as pd
from sklearn.mixture import GaussianMixture as GMM

destination = '';
fileName = os.path.join(destination,'')

imageDir = os.path.join(destination,'normalizedImages')

image_mode1 = 't1'
image_mode2 = 't2'
image_mode3 = 't1c'

concat_t1 = np.empty((0,1))
concat_t2 = np.empty((0,1))
concat_t1c = np.empty((0,1))

with open(fileName)as f:    

    #for row in islice(csv.DictReader(f),0,1):
    for row in csv.DictReader(f):
    
            pid = row['patient_id']
    
            t1_img1 = os.path.join(destination,imageDir,pid,(image_mode1 + '_' + pid + '_norm_rt.nii.gz'))
            t1_img2 = nib.load(t1_img1).get_fdata().reshape(-1,1)
            concat_t1 = np.concatenate((concat_t1,t1_img2))
            
            t2_img1 = os.path.join(destination,imageDir,pid,(image_mode2 + '_' + pid + '_norm_rt.nii.gz'))
            t2_img2 = nib.load(t2_img1).get_fdata().reshape(-1,1)
            concat_t2 = np.concatenate((concat_t2,t2_img2))

            t1c_img1 = os.path.join(destination,imageDir,pid,(image_mode3 + '_' + pid + '_norm_rt.nii.gz'))
            t1c_img2 = nib.load(t1c_img1).get_fdata().reshape(-1,1)
            concat_t1c = np.concatenate((concat_t1c,t1c_img2))
            
    concat_arr = np.hstack((concat_t1,concat_t2,concat_t1c))
    print(concat_arr.shape)
    
    np.savez('t1_t2_t1c_3_norm_tied.npz', t1 = concat_t1, t2 =concat_t2, t1c =concat_t1c)
    
    #decompose our data into 4 Gaussians. n_components=4
    g = GMM(n_components=3,covariance_type='tied')
    g.fit(concat_arr)
    
    weights = g.weights_
    means = g.means_
    print(means)
    covars = g.covariances_
    print(covars)
    
    #save model to file
    gmm_name = 'multi3_norm_tied'
    np.save(gmm_name + '_weights', g.weights_, allow_pickle=False)
    np.save(gmm_name + '_means', g.means_, allow_pickle=False)
    np.save(gmm_name + '_covariances', g.covariances_, allow_pickle=False)

(180391936, 3)
[[2.74790790e-05 3.49282849e-05 4.40508635e-05]
 [1.21548682e+00 4.77694833e+00 1.12207600e+00]
 [1.71479196e+00 2.67530675e+00 1.85517184e+00]]
[[1.05417733e-03 1.16912729e-04 5.83537980e-04]
 [1.16912729e-04 6.72635727e-04 5.16230581e-05]
 [5.83537980e-04 5.16230581e-05 9.48390382e-04]]


In [ ]:
#Predict the cluster in multivariate model
from numba import cuda
cuda.select_device(5)

import os, csv
from itertools import islice

import numpy as np
import nibabel as nib
from sklearn.mixture import GaussianMixture as GMM

from matplotlib.colors import ListedColormap 
import matplotlib.pyplot as plt

destination = '';
fileName = os.path.join(destination,'')

imageDir = os.path.join(destination,'normalizedImages')
outDir = os.path.join(destination,'multivariate')

image_mode1 = 't1'
image_mode2 = 't2'
image_mode3 = 't1c'

# Load Modal
gmm_name = 'multi3_norm_tied'
means = np.load(gmm_name + '_means.npy')
covar = np.load(gmm_name + '_covariances.npy')

loaded_gmm = GMM(n_components = len(means), covariance_type='tied')
loaded_gmm.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
#loaded_gmm.precisions_cholesky_ = np.sqrt(covar) #spherical,diag
loaded_gmm.weights_ = np.load(gmm_name + '_weights.npy')
loaded_gmm.means_ = means
loaded_gmm.covariances_ = covar

with open(fileName)as f: 
    for row in islice(csv.DictReader(f),22,23):
    #for row in csv.DictReader(f):
        
            pid = row['patient_id']
            print(pid)
            
            t1_img1 = os.path.join(destination,imageDir,pid,(image_mode1 + '_' + pid + '_norm_rt.nii.gz'))
            t1_img2 = nib.load(t1_img1) 
            t1_img3 = t1_img2.get_fdata().reshape(-1,1)
            
            t2_img1 = os.path.join(destination,imageDir,pid,(image_mode2 + '_' + pid +'_norm_rt.nii.gz'))
            t2_img2 = nib.load(t2_img1) 
            t2_img3 = t2_img2.get_fdata().reshape(-1,1)

            t1c_img1 = os.path.join(destination,imageDir,pid,(image_mode3 + '_' + pid +'_norm_rt.nii.gz'))
            t1c_img2 = nib.load(t1c_img1) 
            t1c_img3 = t1c_img2.get_fdata().reshape(-1,1)
            
            concat_arr = np.hstack((t1_img3,t2_img3,t1c_img3))
            #print(concat_arr.shape)

            #Predict
            gmm_labels = loaded_gmm.predict(concat_arr)
            #print(gmm_labels.shape)
            original_shape = t1_img2.shape
            segmented = gmm_labels.reshape(original_shape[0], original_shape[1],original_shape[2])
            final_img = nib.Nifti1Image(segmented, t1_img2.affine)
            
            os.mkdir(os.path.join(destination,outDir,pid))
            outputPath = os.path.join(destination,outDir,pid,('multi3_' + pid + '_norm_tied_rt.nii.gz'))
            nib.save(final_img,outputPath)
            